In [14]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from tqdm import tqdm

In [15]:
frequency = 'daily'
column = 'ridership'
years = [2020,2021,2022,2023,2024]

In [16]:
nyc_master_df = pd.read_csv(f'Data/Master Sets/NYC Master/Yearly Combined/NYC_Combined_{frequency}.csv')

In [17]:
nyc_master_df.head()

,Unnamed: 0,year,month,day,station_complex_id,station_complex,borough,latitude,longitude,ridership,...,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,0,2020,10,1,1,"Astoria-Ditmars Blvd (N,W)",Queens,40.775036,-73.912030,5980.0,...,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.912034 40.775036)
1,1,2020,10,1,10,"49 St (N,R,W)",Manhattan,40.759900,-73.984140,4148.0,...,Subway,40.759901,-73.984139,Uptown,Downtown,2.0,1.0,0.0,Uptown only,POINT (-73.984139 40.759901)
2,2,2020,10,1,100,"Hewes St (M,J)",Brooklyn,40.706870,-73.953430,1330.0,...,Elevated,40.706870,-73.953431,Outbound,Manhattan,0.0,0.0,0.0,NaN,POINT (-73.953431 40.70687)
3,3,2020,10,1,101,"Marcy Av (M,J,Z)",Brooklyn,40.708360,-73.957756,5406.0,...,Elevated,40.708359,-73.957757,Outbound,Manhattan,1.0,1.0,1.0,NaN,POINT (-73.957757 40.708359)
4,4,2020,10,1,103,"Bowery (J,Z)",Manhattan,40.720280,-73.993910,1179.0,...,Subway,40.720280,-73.993915,Brooklyn,Downtown,0.0,0.0,0.0,NaN,POINT (-73.993915 40.72028)


In [18]:
nyc_master_df = nyc_master_df[nyc_master_df['year'].isin(years)]

In [19]:
line_list = nyc_master_df['Line'].unique()
line_list.sort()
time_interval = nyc_master_df['unique_time'].unique()
time_interval.sort()

In [20]:
def compute_time_series(col: str):

    if col not in nyc_master_df:
        raise ValueError('Column does not exist')
    temp_dict = {}

    if is_numeric_dtype(nyc_master_df[col]):
        print('is numeric')
    else:
        print('is not numeric')

    for line in tqdm(line_list):
        for time in time_interval:
            temp_df = nyc_master_df[(nyc_master_df['Line'] == line) & (nyc_master_df['unique_time'] == time)]
            if is_numeric_dtype(temp_df[col]):
                val = temp_df[col].sum() # Takes sum of all rows
            else:
                val = temp_df[col].max()
            if line not in temp_dict:
                temp_dict[line] = {}
            temp_dict[line][time] = val
    return temp_dict

In [21]:
ts_matrix = pd.DataFrame(compute_time_series(column))

is numeric


100%|██████████| 32/32 [1:03:45<00:00, 119.54s/it]


In [22]:
ts_matrix

,4th Av,63rd St,6th Av - Culver,8th Av - Fulton St,Astoria,Broadway,Broadway - 7Av,Broadway - Brighton,Canarsie,Clark St,...,Nostrand,Pelham,Queens - Archer,Queens Blvd,Rockaway,Sea Beach,Sea Beach / West End / Culver / Brighton,Second Av,Staten Island,West End
2020-07-01,34912.0,8845.0,60880.0,107356.0,27401.0,32796.0,82224.0,119940.0,47290.0,3110.0,...,14850.0,32621.0,17962.0,86927.0,5966.0,10378.0,4488.0,16038.0,2690.0,16715.0
2020-07-02,36101.0,9229.0,62840.0,113495.0,28760.0,34356.0,85105.0,126405.0,50098.0,3412.0,...,16044.0,32960.0,18441.0,89368.0,7607.0,11200.0,6380.0,16405.0,2630.0,17487.0
2020-07-03,28357.0,6346.0,41504.0,83375.0,19326.0,22827.0,63171.0,92275.0,38246.0,1765.0,...,11944.0,25165.0,13266.0,61785.0,5356.0,8837.0,4875.0,10165.0,1309.0,13523.0
2020-07-04,18820.0,4214.0,22994.0,62007.0,13151.0,15267.0,44755.0,65506.0,27698.0,1805.0,...,8047.0,17494.0,8615.0,38809.0,5412.0,6183.0,6975.0,7091.0,758.0,8287.0
2020-07-05,17669.0,3462.0,20756.0,55562.0,11557.0,13618.0,41577.0,63046.0,24552.0,1408.0,...,6813.0,15808.0,8026.0,35357.0,5336.0,6472.0,5622.0,6067.0,587.0,8624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,99148.0,24427.0,242705.0,330967.0,80108.0,139091.0,268625.0,442485.0,144402.0,13972.0,...,29763.0,46951.0,42016.0,243011.0,12160.0,30398.0,8744.0,39292.0,5879.0,36103.0
2024-12-28,63061.0,16714.0,167382.0,263252.0,56459.0,82291.0,219953.0,356946.0,109536.0,8657.0,...,18509.0,34389.0,26705.0,166461.0,8357.0,17152.0,5526.0,26974.0,2874.0,23947.0
2024-12-29,58967.0,17121.0,164083.0,258621.0,53239.0,82380.0,201162.0,335824.0,105300.0,9009.0,...,16927.0,30817.0,23380.0,152399.0,8048.0,16727.0,5787.0,26135.0,2717.0,24188.0
2024-12-30,95616.0,26147.0,249854.0,349284.0,85010.0,147437.0,284044.0,459649.0,156710.0,14876.0,...,32324.0,49302.0,43605.0,256595.0,12416.0,31511.0,9269.0,43401.0,6425.0,37522.0


In [23]:
# Shortcut solution to set all NA values to 0
ts_matrix.fillna(0, inplace=True)

# TODO Imputation for NA values

In [24]:
ts_matrix.to_csv(f'Data/Master Sets/NYC Master/NYC_Time_Series_{frequency}_{column}_by_Line.csv')